<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/sketch/justin/SPARK-Extract%20Chicago%20from%20Social%20Distancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Chicago Social Distancing Data Using Spark

**Instructions:**

1. Mount the google drive to access the data by following **Steps to Mount the Drive**.
2. Provide the drive file path to the 2020 source data, 2019 source data, the 2020 target directory, 2019 target directory, and the 5 digit combined FIPS codes in the third code cell.

**Steps to Mount the Drive:**

1. Execute the second code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

import os
from tqdm.notebook import tqdm, trange

In [ ]:
# Chicago Counties
fips = ['17031','17037','17043','17063','17091','17089','17093','17111','17197','17097','18073','18089','18111','18127','55059']
f_string = " ('17031','17037','17043','17063','17091','17089','17093','17111','17197','17097','18073','18089','18111','18127','55059') "

path_2020 = 'drive/MyDrive/big-data-project/data/unprocessed/social-distancing/2020/'
target_2020 = 'drive/MyDrive/big-data-project/data/clean-data/ch/social/2020/'

path_2019 = 'drive/MyDrive/big-data-project/data/unprocessed/social-distancing/2019/'
target_2019 = 'drive/MyDrive/big-data-project/data/clean-data/ch/social/2019/'


**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [ ]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 63kB/s 
     |████████████████████████████████| 204kB 37.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=d6b8ba6908a6010ab2b81bd6696ec1ab4bed059b60bbd33f62d3f29b9bf0ede5
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
def filter_fips(df, fips=fips):
    return df[df['cbg'].astype(str).str[:5].isin(fips)]

In [ ]:
def get_fips(fname, filepath, fips, dirout):
    df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(filepath)
    df_soc.createOrReplaceTempView('T')
    df_soc = spark.sql('SELECT * FROM T WHERE SUBSTRING(`origin_census_block_group`,1,5) IN ' + f_string)
    df = df_soc.toPandas()
    df = df.rename(columns={'origin_census_block_group':'cbg'})
    df.to_csv(dirout)

In [ ]:
months = os.listdir(path_2020)
# print(months)
for month in tqdm(months, desc='months'):
    days = os.listdir(path_2020+month)
    days = [day for day in days if day!='.DS_Store']
    for day in tqdm(days, desc='days'):
        sub_path = path_2020 + month + '/' + day +'/'
        f_list = os.listdir(sub_path)
        csv_files = [file for file in f_list if file.endswith('.gz')]
        csv_files = [file for file in csv_files if file!= []]
        for file in csv_files:
            f_path = os.path.join(sub_path, file)
            # Do something with file
            target_dir = os.path.join(target_2020, file)[:-3]  # remove .gz extension
            get_fips(file, f_path, fips, target_dir)

In [ ]:
months = os.listdir(path_2019)
# print(months)
for month in tqdm(months, desc='months'):
    days = os.listdir(path_2019+month)
    days = [day for day in days if day!='.DS_Store']
    for day in tqdm(days, desc='days'):
        sub_path = path_2019 + month + '/' + day +'/'
        f_list = os.listdir(sub_path)
        csv_files = [file for file in f_list if file.endswith('.gz')]
        csv_files = [file for file in csv_files if file!= []]
        for file in csv_files:
            f_path = os.path.join(sub_path, file)
            # Do something with file
            target_dir = os.path.join(target_2019, file)[:-3]  # remove .gz extension
            get_fips(file, f_path, fips, target_dir)